# Improving MCMC Efficiency: QR vs. Basic Parameterizations in Bayesian Linear Regression

## 1. Import Libraries

In [ ]:
import os
import sys

# Add the library directory to the path so Python can find the modules
current_notebook_dir = os.getcwd()
library_dir = os.path.join(current_notebook_dir, "..", "library", "python")
sys.path.append(library_dir)

# Import the needed functions
from generate_data import generate_linear_regression_data
from run_models import run_stan_model_cmdstanpy
from plot import plot_stan_data, plot_trace_plots


## 2. Generate and Plot Data

In [ ]:
N_data = 200
K_features = 5

print(f"--- Generating data for N={N_data}, K={K_features} ---")
stan_data = generate_linear_regression_data(N=N_data, K=K_features, sigma_true=1.5, intercept_true=10.0)

In [ ]:
plot_stan_data(stan_data)

## 3. Models

### 3.1 Basic

The model that we'll fit is:

\begin{align*}
y_{i} &\sim \textrm{Normal}(\mu_{i}, \sigma^{2}) ~~~ \textrm{for}~ i=1, ..., N, \\
\mu_{i} &= \alpha + \mathbf{x}_{i}^{\top} \boldsymbol{\beta}, \\
\alpha &\sim \textrm{Normal}(0, 10^{2}), \\
\boldsymbol{\beta} &\sim \textrm{Normal}(\boldsymbol{0}, 2.5^{2}\mathbf{I}), \\
\sigma &\sim \textrm{Cauchy}(0, 5),
\end{align*}

where 

* $y_{i}$ is the response of the $i$ th observation.
* $\mathbf{x}_{i}$ is the vector of covariate values for the $i$ th observation.
* $\alpha$ is the intercept term.
* $\boldsymbol{\beta}$ is the vector of effects for the fixed covariates.

### 3.2 QR Decomposition


The QR decomposition factorizes a matrix $\mathbf{X} \in \mathbb{R}^{N \times P}$ into two components:

$$\mathbf{X} = \mathbf{Q}\mathbf{R}$$

Where:

* $\mathbf{Q} \in \mathbb{R}^{N \times N}$ is an orthogonal matrix ($\mathbf{Q}^\top \mathbf{Q} = \mathbf{I}$)
* $\mathbf{R} \in \mathbb{R}^{N \times P}$ is an upper triangular matrix

The thin QR decomposition (also called reduced QR decomposition) is more economical when $N > P$ (more observations than predictors):

$$\mathbf{X} = \tilde{\mathbf{Q}}\tilde{\mathbf{R}}$$

Where:

* $\tilde{\mathbf{Q}} \in \mathbb{R}^{N \times P}$ has orthonormal columns ($\tilde{\mathbf{Q}}^\top\tilde{\mathbf{Q}} = \mathbf{I}$)
* $\tilde{\mathbf{R}} \in \mathbb{R}^{P \times P}$ is an upper triangular matrix


In practical applications of thin QR decomposition, especially in Bayesian regression models, the $\tilde{\mathbf{Q}}$ and $\tilde{\mathbf{R}}$ matrices are often scaled by $\sqrt{N-1}$ to produce $\mathbf{Q}_*$ and $\mathbf{R}_*$, such that 

$$\mathbf{X} = \mathbf{Q}_* \mathbf{R}_*$$

Where:

* $\mathbf{Q}_* = \sqrt{N-1} \cdot \tilde{\mathbf{Q}}$
* $\mathbf{R}_* = \frac{1}{\sqrt{N-1}} \cdot \tilde{\mathbf{R}}$

Using the matrix-vector representation of the design matrix/parameter vector, we can write

\begin{align*}
\boldsymbol{\mu} &= \boldsymbol{\alpha} + \mathbf{X} \boldsymbol{\beta} \\
&= \boldsymbol{\alpha} + \mathbf{Q}_* \mathbf{R}_* \boldsymbol{\beta} \\
&= \boldsymbol{\alpha} + \mathbf{Q}_* \boldsymbol{\theta} 
\end{align*}

Letting $\boldsymbol{\theta} = \mathbf{R}_* \boldsymbol{\beta}$. We can also recover $\boldsymbol{\beta}$ with $\boldsymbol{\beta} = \mathbf{R}_*^{-1} \boldsymbol{\theta}$ after sampling.

## 4. Results

In [ ]:
base_path = os.path.join(current_notebook_dir, "..", "library", "stan")

models_to_run = [
    {"name": "Gaussian Linear Regression (Basic)", 
     "path": os.path.join(base_path, "gaussian_linear_regression.stan")},
    {"name": "Gaussian Linear Regression (QR)",
     "path": os.path.join(base_path, "gaussian_linear_regression_QR.stan")}
]

print("\n\n--- RUNNING MODELS ---")
results = {}
for model_info in models_to_run:
    current_data = {k: v for k, v in stan_data.items() if k != "true_params"}
    
    idata = run_stan_model_cmdstanpy(
        model_name=model_info["name"],
        model_code_path=model_info["path"],
        stan_data=current_data
    )
    results[model_info["name"]] = idata

print("\n--- All models processed. ---")

In [ ]:
key_params = ["alpha", "beta", "sigma"]
plot_trace_plots(results, var_names=key_params)